# Design messaging service

### Notes
- FB messenger keep all the messages in DB -> can retrieve the messages
- Whatsapp delete the message on DB after the receiver receive the messages
    + Have end-to-end encryption -> 3rd party cant read the messages
- **http Request**
    + Request - Response architecture: client request - server response
    + Server cant request -> No bidirectional connection 
    + Not suitable for chat/messaging service
- Chat messaging architecture
    + **Websoket**: bidirectional connection
    + **BOSH**: bidirectional stream
    + **long poll http**: bidirectional http

### Features
- User status: Online/ Active [x] hours ago
- Friend adding
- Online 1-on-1 chatting
- Unread/Seen Offline Messages
- Send pictures/files
- Store messages in DB (for searching, etc.)
- Message Encryption

### Architecture
- **Blob Storage**: Store large files/data (images, text, files, etc.)
- **Node**: Manage users

<img src="./img/5.jpg" alt="drawing" width="650"/>

### Database schema
- **Friend Table**

| Conversation_id | User 1 | User 2 | Encryption_key |
|:---------------:|:------:|:------:|:--------------:|
|        5        | User_A | User_B |     AB152C1    |
|        6        | User_A | User_C |     C458A29    |

- **Conversation Table**

| Conversation_id |  Time |         Text         | url                   | From_user | To_user |
|:---------------:|:-----:|:--------------------:|:---------------------:|:---------:|:-------:|
|        5        | 12:00 |        "Hi"          | None                  |User_A     | User_B  |
|        5        | 12:01 |"Hi, Nice to meet you"| None                  |User_B     | User_A  |
|        6        | 13:04 |      "Hello"         | None                  |User_A     | User_C  |
|        5        | 15:13 |        None          | fbimg.com/45fdfC      |User_A     | User_B  |

- **Unread Table**

| From_user | To_user |  time |
|:---------:|:-------:|:-----:|
|   User_A  |  User_C | 13:04 |

- **Seen Table**

| From_user | To_user | time  |
|:---------:|:-------:|:-----:|
|   User_A  |  User_C | 14:09 |



### Design Features

#### User status: Online/ Active [x] hours ago
- User A login through Authentication system
- User A connects bidirectional with Node 2 through Load balancer
    + N2 manages User A
    + User A continously sends heartbeating to N2
    + Info stored in **Distributed cache** and kept updated by N2

| User   | Server | Heartbeat time     |
|:------:|:------:|:------------------:|
| User A | N2     | Mar 13 15:51:22    |
| User B | N3     | Mar 13 15:51:22    |
| User C | N1     | Mar 13 13:41:31    |

- Based on the heartbeat time -> N2 inform User A about his friends status: online or Active [x] hours ago

```
User A friend list:
    User B: Online
    User C: Active 2 hours ago
```
<img src="./img/4.jpg" alt="drawing" width="850"/>

#### Friend Adding and Security
- If A and B become friends
- put on **Friend Table** 
- Conversation_id is UNIQUE
- Encryption_key is used to encrypt/decrypt the messages between A <-> B


#### Online 1-on-1 chatting
- User A and B are friends
- User A online heartbeating N2, User B online heartbeating N3

```
    User A: Online
    User B: Online
```

- User A want to send messages to User B
    + User A send message to N2
    + N2 put the message onto **Conversation Table**
    + N2 check if any node manages user B -> Found N3
    + N2 send message to N3
    + N3 sends messages to User B

<img src="./img/6.jpg" alt="drawing" width="850"/>

#### Offline Messages
- User A and C are friends
- User A online and heartbeating N2, User C offline

```
    User A: Online
    User C: Active 2 hrs ago
```

- User A send a message to User C offline
    + **13:04** User A send message to N2, N2 put the message onto DB **Conversation Table**
    + N2 check if any node manages user C -> Not Found
    + N2 puts a note onto **Unread Table**, inform User A message is sent and unread

- User C online
    + **14:09** User C online, heartbeating to N1
    + N1 check **Unread Table**, retrieve the message from **Conversation Table** to User C
    + N1 remove the entry from **Unread Table**, puts a note onto **Seen Table**, inform N2 that User C read the message
    + N2 check the **Seen Table**, inform the read time to User A and remove the entry from **Seen Table**
    
<img src="./img/7.jpg" alt="drawing" width="850"/>

#### Send pictures/files
- User A and B are friends
- User A online heartbeating N2, User B online heartbeating N3

```
    User A: Online
    User B: Online
```

- User A sends a picture to User B
    + User A sends pic to N2
    + N2 store pic into **Blob storage** and get back the url
    + N2 put the url into **Conversation Table**
    + N2 check if any node manages user B -> Found N3
    + N2 sends url and thumbnail to N3
    + N3 sends url and thumbnail -> User B
    + User B can download pic through url

<img src="./img/8.jpg" alt="drawing" width="850"/>

### Optimizations
- Cache old conversation data -> text file -> store into Blob storage